---
# Нормализация извеченных сущностей

> **🎯 Цель этого ноутбука:** Нормализвать сущности, извлеченные при помощи модели, обученной на предыдущем этапе. Нормализаиция - процесс приведения текстовых упоминаний к стандартному виду 
---

## Импорт библиотек

In [1]:
import pandas as pd
import pymorphy3

## Нормализация

In [2]:
df = pd.read_parquet("../data/chgk_entities_structured.parquet")

In [3]:
df

,id,author,date,entity_group,score,word,start,end
0,420110,Андрей Скиренко,2023-11-05 16:00:00,LOCATION,0.998974,мексики,251,258
1,420111,Дмитрий Рыжков,2023-11-05 16:00:00,PERSON,0.999130,дэвид теннант,19,32
2,420111,Дмитрий Рыжков,2023-11-05 16:00:00,PERSON,0.999153,маикл шин,35,44
3,420111,Дмитрий Рыжков,2023-11-05 16:00:00,PERSON,0.998630,шина дэвид,98,108
4,420111,Дмитрий Рыжков,2023-11-05 16:00:00,PERSON,0.999093,маикл шин,192,201
...,...,...,...,...,...,...,...,...
1129744,18785,Алексей Паевский,2000-09-02 00:00:00,AUTHOR,0.583950,верна,134,139
1129745,18785,Алексей Паевский,2000-09-02 00:00:00,PERSON,0.515345,##дс,139,141
1129746,18785,Алексей Паевский,2000-09-02 00:00:00,AUTHOR,0.544298,##кии,141,144
1129747,18785,Алексей Паевский,2000-09-02 00:00:00,AUTHOR,0.675261,маяковскии,146,156


In [4]:
is_punctuation_only = df['word'].str.match(r'^[\W_]+$', na=False)
contains_hash = df['word'].str.contains('#', na=False)
df_cleaned = df[~(is_punctuation_only | contains_hash)]

In [5]:
df_cleaned['word'] = df_cleaned['word'].apply(lambda x: x.strip(' \'\"«»”`‘‚’‹›'))

/var/folders/v0/x45rbc1s671ft5z86pdqjpvw0000gn/T/ipykernel_79420/444093890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['word'] = df_cleaned['word'].apply(lambda x: x.strip(' \'\"«»”`‘‚’‹›'))


In [6]:
# 1. Создаем анализатор (лучше один раз, а не в цикле)
morph = pymorphy3.MorphAnalyzer()
df_cleaned['word'] = df_cleaned['word'].apply(lambda x: morph.parse(x)[0].normal_form)

/var/folders/v0/x45rbc1s671ft5z86pdqjpvw0000gn/T/ipykernel_79420/1112434147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['word'] = df_cleaned['word'].apply(lambda x: morph.parse(x)[0].normal_form)


In [7]:
df_cleaned.drop_duplicates(['id', 'entity_group', 'word'], inplace=True)

/var/folders/v0/x45rbc1s671ft5z86pdqjpvw0000gn/T/ipykernel_79420/2070732051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop_duplicates(['id', 'entity_group', 'word'], inplace=True)


In [8]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 941696 entries, 0 to 1129748
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            941696 non-null  int64  
 1   author        903331 non-null  object 
 2   date          941696 non-null  object 
 3   entity_group  941696 non-null  object 
 4   score         941696 non-null  float32
 5   word          941696 non-null  object 
 6   start         941696 non-null  int64  
 7   end           941696 non-null  int64  
dtypes: float32(1), int64(3), object(4)
memory usage: 61.1+ MB


In [9]:
df_cleaned.to_csv("../data/final_ent.csv")

## Примечание

От качетва этого этапа в значительной степени зависит качетво дальнейшего анализа. В дальнейшем планируется выполнить данный этап с использованием более продвинутых техник, что позволит значительно увеличить качетво анализа.